In [3]:
import roboidai as ai
import cv2
from ultralytics import YOLO
import yaml
import os
!pwd 
# Train Setting
train_mode = False
image_size = 1280
epoch = 10
batch_size = 8
train_data = os.getcwd() + '/dataset/data.yaml'
#'/home/yumjunstar/Desktop/YJW/dataset/data.yaml'

# Target Camera
camera_target = 'ip0'

# Load label list
all_child_names = [] # 모든 class의 목록
with open(train_data) as f:
    loaded_value = yaml.safe_load(f)
    all_child_names = loaded_value['names']
    print ('All child names', all_child_names)
# Childeren Target Setting

target_child_name = "face" # 우리가 찾으려는 아이의 이름
target_child_id = all_child_names.index(target_child_name) # 이름을 컴퓨터가 읽을 수 있는 index로 변환

# Camera Target Setting
error_x = 0
allowed_error = 0.1

model = YOLO('runs/detect/train57_version7/best.pt')

if train_mode:
    history = model.train(data = train_data, imgsz = image_size, epochs = epoch, batch = batch_size)
else:
    model = YOLO('runs/detect/train57_version7/best.pt')




cam = ai.Camera(camera_target)


while True:
    img = cam.read() # 카메라 열기
    print (img)
    pred_result = list(model(img)) # 예측된 결과 모두 불러오기

    get_result_img = pred_result[0].plot() # 예측된 결과 이미지 그리기
    get_boxes = pred_result[0].boxes # 예측된 결과의 박스 좌표 가져오기

    for i, box in enumerate(get_boxes): # 모든 박스 불러오기
        print (box.cls) # 박스의 클래스 출력
        if box.cls == target_child_id:
            print (box.xywhn)
            center_x, center_y, width, height = list(box.xywhn[0])
            print ('{name} childeren found'.format(name = target_child_name), center_x, center_y, width, height)
            print ('Robot moving')
            error_x = 0.5 - center_x #목표값 - 현재 값
            
            if abs(error_x) <= allowed_error:
                print ("{name} childeren infront of me".format(name = target_child_name))
                #Robot Stop
            break

    cv2.imshow("YOLOv8 Inference", get_result_img)
    if cam.check_key() == 'esc': break

/home/yumjunstar/Desktop/YJW/RoboticsProgram/Project/LostChildProject-ROS2_SLAM_Toolbox-Yolov8/Vision_immature
All child names ['ben_afflek', 'elton_john', 'face', 'jerry_seinfeld', 'madonna', 'mindy_kaling']
scanning ip camera...


WARNING ⚠️ 'source' is missing. Using 'source=https://ultralytics.com/images/bus.jpg'.


scanning completed
None



Found https://ultralytics.com/images/bus.jpg locally at bus.jpg


KeyboardInterrupt: 

In [1]:
import numpy as np
import cv2
import glob
import roboidai as ai

camera_target = 'ip0'
cam = ai.Camera(camera_target)

images = []

capture_count = 0
while capture_count < 30:
    img = cam.read()
    cam.show(img)
    key_input = cam.check_key()
    if key_input == 'esc':
        break
    elif key_input == ' ':
        print ('camera captured')
        images.append(img)
        capture_count += 1
        cv2.imshow(str(capture_count), img)
        cv2.imwrite(f"{capture_count}.png", img)

def camera_calibration_value (images):
    # termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*7,3), np.float32)
    objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.
    gray = []


    for index_, img in enumerate(images):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, (7,6),None)
        # If found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
            imgpoints.append(corners2)
            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)
            cv2.imshow('img_processed {index}'.format(index = index_),img)
        else:
            print ('NOt found')
        cv2.waitKey(500)
    print (gray.shape[::-1])

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

    return ret, mtx, dist,rvecs,tvecs

ret, mtx, dist, rvecs, tvecs = camera_calibration_value(images)
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
x,y,w,h = roi
while True:
    img = cam.read()


    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    dst = dst[y:y+h, x:x+w]
    cv2.imshow('calibresult',dst)


cv2.destroyAllWindows()

2023-06-16 18:00:08.966932: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-16 18:00:09.405974: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-16 18:00:09.408370: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 18:00:12.196068: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


SystemExit: 

/home/yumjunstar/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
import cv2
import numpy as np

# 카메라 매개변수 설정
rms = 0.628610
fx = 389.438894
fy = 389.438894
cx = 320.000000
cy = 160.000000
k1 = -0.357859
k2 = 0.142288
p1 = -0.010197
p2 = -0.001690

hfov = 78.8
vfov = 44.7

# 카메라 매개변수로 카메라 행렬 생성
camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

# 왜곡 계수 생성
dist_coeffs = np.array([k1, k2, p1, p2])

# 왜곡 보정에 사용할 이미지 로드
image = cv2.imread('30.png')

# 카메라 행렬과 왜곡 계수를 사용하여 왜곡 보정
undistorted_image = cv2.undistort(image, camera_matrix, dist_coeffs)

# 보정된 이미지 출력
cv2.imshow('Undistorted Image', undistorted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
import cv2
import numpy as np

# 카메라 매개변수 설정
rms = 0.628610
fx = 389.438894
fy = 389.438894
cx = 320.000000
cy = 160.000000
k1 = -0.357859
k2 = 0.142288
p1 = -0.010197
p2 = -0.001690
k3 = 0.0  # 추가: k3 왜곡 계수

hfov = 78.8
vfov = 44.7

# 카메라 매개변수로 카메라 행렬 생성
camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

# 왜곡 계수 생성
dist_coeffs = np.array([k1, k2, p1, p2, k3])  # 수정: k3 왜곡 계수 추가

# 왜곡 보정에 사용할 이미지 로드
image = cv2.imread('28.png')

# 이미지 크기에 맞는 새로운 카메라 매개변수 계산
new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, (image.shape[1], image.shape[0]), 1)

# 보정된 이미지 계산
undistorted_image = cv2.undistort(image, camera_matrix, dist_coeffs, None, new_camera_matrix)

# ROI를 사용하여 보정된 이미지를 적절한 크기로 잘라냄
x, y, w, h = roi
undistorted_image = undistorted_image[y:y+h, x:x+w]

# 보정된 이미지 출력
cv2.imshow('Undistorted Image', undistorted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()